In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import matplotlib as mpl
import catppuccin
from keras.utils import to_categorical
from keras.layers import Rescaling

mpl.style.use(catppuccin.PALETTE.mocha.identifier)

(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()

X_train_grayscaled = np.dot(X_train[..., :3], [0.2989, 0.5870, 0.1140])
X_test_grayscaled = np.dot(X_test[..., :3], [0.2989, 0.5870, 0.1140])

y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)
normalization_layer = Rescaling(1./255)
X_train_grayscaled = normalization_layer(X_train_grayscaled)
X_test_grayscaled = normalization_layer(X_test_grayscaled)

In [ ]:
# Baseline model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Rescaling

model = Sequential([
    Rescaling(1./255),
    # First convolutional layer with 32 filters, 3x3 kernel, ReLU activation
    # Input shape is 32x32 pixels with 3 color channel (grayscale)
    Conv2D(32, (3,3), activation='relu', input_shape=(32, 32, 3)),
    # Max pooling layer to reduce spatial dimensions by a factor of 2
    MaxPooling2D(2,2),
    # Second convolutional layer with 64 filters, 3x3 kernel, ReLU activation
    Conv2D(64, (3,3), activation='relu'),
    # Another max pooling layer to reduce size further
    MaxPooling2D(2,2),
    # Flatten the 2D feature maps into a 1D vector for dense layers
    Flatten(),
    # Fully connected (dense) layer with 128 neurons and ReLU activation
    #Learns intermediate features from the previous layers
    Dense(128, activation='relu'),
    # Output layer with 10 neurons (for 10 classes) and softmax activation
    #Produces the final probabilities for each class.
    Dense(10, activation='softmax')
])

model.summary()

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(
    X_train,
    y_train,
    epochs=20,
    validation_data=(X_test, y_test),
)
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy of baseline model: {accuracy:.4f}")

In [ ]:
model.save("initial_model.keras")

In [ ]:
# the other (more complex) model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam, Nadam, SGD

In [ ]:
model_more_layers = Sequential([
    Rescaling(1./255),
    Conv2D(32, (3,3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Dropout(0.4),
    Flatten(),
    Dense(128, activation='relu'),
    Flatten(),
    #Produces the final probabilities for each class.
    Dense(10, activation='softmax'),
])

model_more_layers.summary()

In [ ]:

learning_rate = 1e-2
epochs = 300

adam = Adam(
    learning_rate=learning_rate,
)
nadam = Nadam(
    learning_rate=learning_rate,
)
sgd = SGD(
    learning_rate=learning_rate,
    nesterov=True,
)

model_more_layers.compile(
    optimizer=sgd,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    patience=6,
    min_lr=1e-4,
)

# Train on original data
history_more_layers = model_more_layers.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping, reduce_lr],
)

# Evaluate
loss, accuracy = model_more_layers.evaluate(X_test, y_test)
print(f"Test accuracy with more layers: {accuracy:.4f}")

In [ ]:
model_more_layers.save("model_with_more_layers.keras")

In [ ]:
# transfer learning model 1
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, InputLayer, Resizing
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam, Nadam, SGD, Adadelta
from keras.applications import EfficientNetB0, EfficientNetV2S, EfficientNetV2B0, MobileNetV3Large, MobileNetV2
import tensorflow as tf

In [ ]:
base_model = MobileNetV3Large(
    include_top=False,
    weights='imagenet',
)
base_model.trainable=False

model = Sequential([
    Resizing(32*5, 32*5),
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax'),
])

adadelta = Adadelta(learning_rate=0.1)
model.compile(
    optimizer=adadelta,
    loss='sparse_categorical_crossentropy',
    metrics=["accuracy"],
)

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    patience=6,
    min_lr=1e-4,
)

history = model.fit(
    X_train, y_train,
    epochs=300,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr],
    validation_data=[X_test, y_test]
)

est_loss, test_acc = model.evaluate(tf.image.resize(X_test, (32*5, 32*5)), y_test)
print(f"Test Accuracy: {test_acc:.4f}")

In [ ]:
model.save("transfer-learning1.keras")

In [ ]:
# transfer learning model 2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, InputLayer
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam, Nadam, SGD, Adadelta
from keras.applications import EfficientNetB0, EfficientNetV2S, EfficientNetV2B0, MobileNetV3Large, MobileNetV2
import tensorflow as tf

In [ ]:
base_model = MobileNetV3Large(
    include_top=False,
    weights='imagenet',
)

image_resize_factor = 5
base_model.trainable=False
fraction_retrained_layers = .3
layer_count = len(base_model.layers)
layers_to_retrain = round(layer_count * fraction_retrained_layers)
for layer in base_model.layers[-layers_to_retrain:]:
    layer.trainable = True

model = Sequential([
    Resizing(32*image_resize_factor, 32*image_resize_factor),
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax'),
])

adadelta = Adadelta(learning_rate=0.1)
model.compile(
    optimizer=adadelta,
    loss='sparse_categorical_crossentropy',
    metrics=["accuracy"],
)

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    patience=6,
    min_lr=1e-4,
)

history = model.fit(
    X_train, y_train,
    epochs=300,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr],
    validation_data=[tf.image.resize(X_test, (32*image_resize_factor, 32*image_resize_factor)), y_test]
)

est_loss, test_acc = model.evaluate(tf.image.resize(X_test, (32*image_resize_factor, 32*image_resize_factor)), y_test)
print(f"Test Accuracy: {test_acc:.4f}")

In [ ]:
model.save("./transfer-learning2-with-partial-retrain-of-base-model.keras")